# MGTA 415 - Session 0 Demo: A simple example of paper trading with Alpaca API

This notebook demonstrates how to use the Alpaca API to pull real-time news, analyze sentiment, and make trading decisions based on the sentiment analysis.

## Step 1: Import Libraries

First, we'll import the necessary libraries and import the secrets. 
This assumes the secrets are already added in .env file.  Everytime you update the .env file you'll need to rebuild the container.

In [10]:
import os
import alpaca_trade_api as tradeapi
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta
import requests
import pandas as pd

In [4]:
ALPACA_URL = 'https://paper-api.alpaca.markets'
SYMBOLS = ["AAPL"]

In [5]:
# Alpaca API credentials
ALPACA_API_KEY = "PKDFQIPQCXE38TAG1WKG"
ALPACA_SECRET_KEY = "LRaWwamsMm0WDa58x8S0z8wje9gRcYhcgtMf7C55"

# Initialize Alpaca API
alpaca = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url=ALPACA_URL)

# Initialize Sentiment Analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

## Step 2: Retrieve News from Previous Day

We'll retrieve news articles from the previous day using Alpaca's API.


In [11]:
# Function to retrieve news from the previous day for specific symbols
def get_previous_day_news(symbols):
    today = datetime.utcnow()
    yesterday = today - timedelta(days=30)
    url = "https://data.alpaca.markets/v1beta1/news"
    headers = {
        "APCA-API-KEY-ID": ALPACA_API_KEY,
        "APCA-API-SECRET-KEY": ALPACA_SECRET_KEY
    }
    params = {
        "symbols": ",".join(symbols),
        "start": yesterday.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "end": today.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "limit": 50
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()['news']

# Retrieve news for specific symbols
news = get_previous_day_news(SYMBOLS)

news_df = pd.DataFrame(news)

In [19]:
from newspaper import Article

def get_article_content(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"Error fetching content from {url}: {e}")
        return ""

# Assuming 'news' is a list of dictionaries where each dictionary contains a 'url' key
for entry in news:
    if 'url' in entry:
        entry['content'] = get_article_content(entry['url'])

# Now, each dictionary in the 'news' list will have a 'content' key with the extracted article content


Error fetching content from https://www.benzinga.com/news/24/08/40424112/apple-reportedly-has-several-hundred-working-on-a-robot-arm-with-attached-ipad-techcrunch: Article `download()` failed with 429 Client Error: Too Many Requests for url: https://www.benzinga.com/news/24/08/40424112/apple-reportedly-has-several-hundred-working-on-a-robot-arm-with-attached-ipad-techcrunch on URL https://www.benzinga.com/news/24/08/40424112/apple-reportedly-has-several-hundred-working-on-a-robot-arm-with-attached-ipad-techcrunch
Error fetching content from https://www.benzinga.com/news/24/08/40375642/apple-moves-ahead-with-tabletop-robot-push-led-by-ex-car-chief: Article `download()` failed with 429 Client Error: Too Many Requests for url: https://www.benzinga.com/news/24/08/40375642/apple-moves-ahead-with-tabletop-robot-push-led-by-ex-car-chief on URL https://www.benzinga.com/news/24/08/40375642/apple-moves-ahead-with-tabletop-robot-push-led-by-ex-car-chief
Error fetching content from https://www.ben

## Step 3: Analyze Sentiment and Make Trading Decisions

We'll analyze the sentiment of the news articles and make trading decisions based on the sentiment scores.


In [23]:
# Function to analyze sentiment and make trading decisions
def analyze_and_trade(news):
    for article in news:
        headline = article['headline']
        content = article.get('content', '')
       
        
        sentiment_headline = sentiment_analyzer.polarity_scores(headline)
        sentiment_content = sentiment_analyzer.polarity_scores(content)
        sentiment_score_headline = sentiment_headline['compound']
        sentiment_score_content = sentiment_content['compound']
        sentiment_score = (sentiment_score_headline + sentiment_score_content) / 2
        print(f"Headline: {headline}")
        print(f"Sentiment Score: {sentiment_score}")
        

In [2]:
# Function to make trading decisions
def make_trade_decision(symbol, action):
    qty = 10  # Example quantity
    
    try:
        if action == 'buy':
            alpaca.submit_order(
                symbol=symbol,
                qty=qty,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
            print(f"Buying {qty} shares of {symbol}")
        elif action == 'sell':
            alpaca.submit_order(
                symbol=symbol,
                qty=qty,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
            print(f"Selling {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error placing order: {e}")




NameError: name 'news' is not defined

In [24]:
# Analyze and trade based on the news
analyze_and_trade(news)

Headline: 10 Information Technology Stocks Whale Activity In Today's Session
Sentiment Score: 0.4978
Headline: Stock Market Rebounds To Resistance Zone, AMD's AI Strategy To Compete With Nvidia
Sentiment Score: 0.49885
Headline: MoffettNathanson Initiates Coverage On Apple with Neutral Rating, Announces Price Target of $211
Sentiment Score: 0.4722
Headline: "Sources Say Apple's In-House Modem Chip Implementation Is Slow; Is The Release Of Apple's Proprietary Modem Imminent? According To Industry Sources, The Process Of Implementing It Will Remain Gradual In 2025, With Existing Chip Provider Qualcomm Not Significantly Affected" - DigiTimes Alert
Sentiment Score: 0.66755
Headline: Wearable Devices' Mudra Band Supports Apple Vision Pro Gesture Controls To All Apple Devices
Sentiment Score: 0.74335
Headline: Taiwan Semiconductor Acquires Tainan Facility For $531M, Plans Major Expansion
Sentiment Score: 0.4899
Headline: Warren Buffett's $189B Cash Pile Not A Sign Of Looming Market Crash, Sa